In [186]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest,chi2
from sklearn import set_config 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [187]:
set_config(display="diagram")

In [188]:
df=pd.read_csv("./datasets/train.csv")

In [189]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [190]:
df=df[["Pclass","Age","Fare","Survived"]]

In [191]:
df.shape

(891, 4)

In [192]:
df.head(5)

,Pclass,Age,Fare,Survived
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [193]:
df.duplicated().sum()

145

In [194]:
df.drop_duplicates(inplace=True)

In [195]:
df.shape

(746, 4)

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 746 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    746 non-null    int64  
 1   Age       658 non-null    float64
 2   Fare      746 non-null    float64
 3   Survived  746 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 29.1 KB


Age--> having the missing value

In [197]:
df.isnull().mean()*100

Pclass       0.000000
Age         11.796247
Fare         0.000000
Survived     0.000000
dtype: float64

In [198]:
df.describe()

,Pclass,Age,Fare,Survived
count,746.000000,658.000000,746.000000,746.000000
mean,2.227882,29.841064,34.787785,0.416890
std,0.860201,14.799527,50.177031,0.493375
min,1.000000,0.420000,0.000000,0.000000
25%,1.000000,20.000000,8.050000,0.000000
50%,3.000000,28.250000,16.100000,0.000000
75%,3.000000,39.000000,34.913550,1.000000
max,3.000000,80.000000,512.329200,1.000000


In [199]:
df.corr()

,Pclass,Age,Fare,Survived
Pclass,1.000000,-0.368138,-0.565326,-0.331683
Age,-0.368138,1.000000,0.094089,-0.084596
Fare,-0.565326,0.094089,1.000000,0.247510
Survived,-0.331683,-0.084596,0.247510,1.000000


# train_test_split

In [200]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,:3],df.iloc[:,3:4])

In [201]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((559, 3), (187, 3), (559, 1), (187, 1))

In [202]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559 entries, 730 to 668
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  559 non-null    int64  
 1   Age     498 non-null    float64
 2   Fare    559 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 17.5 KB


# Pipeline Using KNNImputer

In [245]:
trf1=ColumnTransformer(transformers=[
    ('KNNImputer',KNNImputer(weights="distance",n_neighbors=5
                            ),[0,1,2])
],remainder="passthrough")

In [246]:
trf2=ColumnTransformer(transformers=[
    ('scale',MinMaxScaler(),slice(0,3))
])

In [247]:
trf3=SelectKBest(score_func=chi2,k=3)

In [248]:
trf4=LogisticRegression()

In [249]:
pipe=Pipeline([('trf1',trf1),('trf2',trf2),('trf3',trf3),('trf4',trf4)])

In [250]:
pipe.fit(X_train,y_train)

/home/ajit/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('KNNImputer',
                                                  KNNImputer(weights='distance'),
                                                  [0, 1, 2])])),
                ('trf2',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 3, None))])),
                ('trf3',
                 SelectKBest(k=3,
                             score_func=<function chi2 at 0x7f2643dc6040>)),
                ('trf4', LogisticRegression())])

In [251]:
y_pred=pipe.predict(X_test)

In [252]:
accuracy_score(y_pred,y_test)

0.6524064171122995

In [253]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

/home/ajit/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ajit/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ajit/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ajit/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

0.7011904761904761

# Pipeline using SimpleImputer

In [212]:
trf1=ColumnTransformer(transformers=[
    ('KNNImputer',SimpleImputer(),[1])
],remainder="passthrough")

In [213]:
trf2=ColumnTransformer(transformers=[
    ('scale',MinMaxScaler(),slice(0,3))
])

In [214]:
trf3=SelectKBest(score_func=chi2,k=3)

In [215]:
trf4=LogisticRegression()

In [216]:
pipe=Pipeline([('trf1',trf1),('trf2',trf2),('trf3',trf3),('trf4',trf4)])

In [217]:
pipe.fit(X_train,y_train)

/home/ajit/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('KNNImputer', SimpleImputer(),
                                                  [1])])),
                ('trf2',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 3, None))])),
                ('trf3',
                 SelectKBest(k=3,
                             score_func=<function chi2 at 0x7f2643dc6040>)),
                ('trf4', LogisticRegression())])

In [218]:
y_pred=pipe.predict(X_test)

In [219]:
accuracy_score(y_pred,y_test)

0.6524064171122995

In [220]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

/home/ajit/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ajit/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ajit/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ajit/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape

0.6958333333333334